In [27]:
import pandas as pd
import requests
from splinter import Browser
from bs4 import BeautifulSoup
import time

In [28]:
# Yelp url for restaurants in Morris County, NJ to be scraped
url = 'https://www.yelp.com/search?find_desc=Restaurants&find_loc=Morris+County%2C+NJ&ns=1'
response = requests.get(url)

In [29]:
# intialize variables and create empty lists
nextpage_link = True
restaurant_name = []
restaurant_url = []
restaurant_rating = []
price_cuisine = []
 
while (nextpage_link):
    
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # class for restaurant details
    containers = soup.find_all('div', class_='scrollablePhotos__09f24__1PpB8')[1:]
    
    # scroll through each container
    for item in containers:
        #print(item)
        # find and add restaurant name to list
        rest_name = item.find('a',class_='link__09f24__1kwXV')
        restaurant_name.append(rest_name.text)
        
        # find and add restaurant url to list
        restaurant_url.append("https://www.yelp.com" + rest_name['href'])
        
        # find and add rating (if exists) to list
        rating = item.find('div',class_='i-stars__09f24__1T6rz')
        if rating:
            restaurant_rating.append(rating['aria-label'].split(' ')[0])
        else:
            restaurant_rating.append("")
        
        # find and add cuisine to list
        price_cuisine.append(item.find('div',class_='priceCategory__09f24__2IbAM').text)

    # find the next page url
    next_page = soup.find('a', class_='next-link')
    
    # if there is a another page, assign the url, otherwise exit
    if next_page:        
        nextpage_url = next_page['href']
        response = requests.get(nextpage_url)
        #time.sleep(2)
    else:
        nextpage_link = False

# Create Dataframe
yelp_df = pd.DataFrame({'restaurant_name': restaurant_name,
                       'restaurant_url': restaurant_url,
                       'restaurant_rating': restaurant_rating,
                       'price_cuisine': price_cuisine})

yelp_df    

,restaurant_name,restaurant_url,restaurant_rating,price_cuisine
0,SubUrban Bar & Kitchen,https://www.yelp.com/biz/suburban-bar-and-kitc...,4,"$$American (New), Bars, Pizza"
1,Rosie’s Trattoria,https://www.yelp.com/biz/rosie-s-trattoria-ran...,4.5,"$$$Italian, Bars, Venues & Event Spaces"
2,4 Seasons Mediterranean Restaurant,https://www.yelp.com/biz/4-seasons-mediterrane...,4.5,"$$Wine Bars, French, Italian"
3,Quiet Man Pub,https://www.yelp.com/biz/quiet-man-pub-dover-2...,4.5,"$$Irish Pub, Seafood, Tapas/Small Plates"
4,ALEV Mediterranean Grill,https://www.yelp.com/biz/alev-mediterranean-gr...,4.5,"$$Mediterranean, Turkish, Kebab"
...,...,...,...,...
235,Los Criollos,https://www.yelp.com/biz/los-criollos-wharton?...,3.5,$$Latin American
236,Weenies,https://www.yelp.com/biz/weenies-denville?osq=...,4,$Hot Dogs
237,Monchy’s Colombian Grill,https://www.yelp.com/biz/monchys-colombian-gri...,2.5,$Colombian
238,My Kitchen,https://www.yelp.com/biz/my-kitchen-rockaway?o...,4,"Buffets, American (Traditional)"


In [43]:
# create empty lists to hold address
address1 = []
address2 = []
restaurant_website = []

executable_path = {"executable_path": "/usr/local/bin/chromedriver"}
browser =  Browser("chrome", **executable_path, headless=False)

# iterate through all the rows in the yelp dataframe
for index, row in yelp_df.iterrows():
    
    # visit each restaurant's url
    browser.visit(row['restaurant_url'])

    html = browser.html
    soup = BeautifulSoup(html, 'html.parser')
    
    # find the address tag
    address = soup.find('address').find_all('span', class_='raw__373c0__3rcx7')
    
    # find address1 and add to list
    street_address = address[0].text
    for i in range(1, len(address)-1):
        street_address = street_address + address[i].text + " "
    address1.append(street_address)
    
    # find the rest of the address
    address2 = address[len(address)-1].text
    
    rest_website = soup.find('div', class_='stickySidebar__373c0__3PY1o').find('a', class_='lemon--a__373c0__IEZFH').text

    if rest_website == "Get Directions":
        restaurant_website.append("")
    else:
        restaurant_website.append(rest_website)
    
browser.quit()

# add address columns to yelp dataframe
yelp_df['address1'] = address1
yelp_df['address2'] = address2
yelp_df['restaurant_url'] = restaurant_website
                  
yelp_df

AttributeError: 'NoneType' object has no attribute 'find_all'

In [5]:
# save to csv file
yelp_df.to_csv('../Resources/yelp_data.csv')

In [9]:
# USING REQUESTS
# create empty lists to hold address
street = []
city = []
state = []
zipcode = []

# iterate through all the rows in the yelp dataframe
for index, row in yelp_df.iterrows():
    
    # visit each restaurant's url
    print(row['restaurant_url'])
    rest_response = requests.get(row['restaurant_url'])
    time.sleep(2)

    rest_soup = BeautifulSoup(rest_response.text, 'html.parser')
    
    # address_tag = rest_soup.find_all('address', class_='lemon--address__373c0__2sPac')
    address_tag = rest_soup.find_all('section', class_='lemon--section__373c0__fNwDM')
    
    print(address_tag)
    
    # find the address tag
    address = soup.find('address').find_all('span', class_='raw__373c0__3rcx7')
    
    # find address1 and add to list
    street_address = address[0].text
    for i in range(1, len(address)-1):
        street_address = street_address + address[i].text + " "
    address1.append(street_address)
    
    # find the rest of the address
    address2 = address[len(address)-1].text

   
    print(address)
    
    
yelp_df['street'] = street
yelp_df['city'] = city
yelp_df['state'] = state
yelp_df['zipcode'] = zipcode
                  
yelp_df

https://www.yelp.com/biz/rosie-s-trattoria-randolph?osq=Restaurants
[]


AttributeError: ResultSet object has no attribute 'find_all'. You're probably treating a list of elements like a single element. Did you call find_all() when you meant to call find()?

In [30]:
json_file = "../Resources/yelp_academic_dataset_business.json"
yelp_original_df = pd.read_json(json_file, lines=True)
yelp_original_df.head()

#yelp_original_df.groupby('state')['state'].count().sort_values(ascending=False)

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
0,f9NumwFMBDn751xgFiRbNA,The Range At Lake Norman,10913 Bailey Rd,Cornelius,NC,28031,35.462724,-80.852612,3.5,36,1,"{'BusinessAcceptsCreditCards': 'True', 'BikePa...","Active Life, Gun/Rifle Ranges, Guns & Ammo, Sh...","{'Monday': '10:0-18:0', 'Tuesday': '11:0-20:0'..."
1,Yzvjg0SayhoZgCljUJRF9Q,"Carlos Santo, NMD","8880 E Via Linda, Ste 107",Scottsdale,AZ,85258,33.569404,-111.890264,5.0,4,1,"{'GoodForKids': 'True', 'ByAppointmentOnly': '...","Health & Medical, Fitness & Instruction, Yoga,...",None
2,XNoUzKckATkOD1hP6vghZg,Felinus,3554 Rue Notre-Dame O,Montreal,QC,H4C 1P4,45.479984,-73.580070,5.0,5,1,None,"Pets, Pet Services, Pet Groomers",None
3,6OAZjbxqM5ol29BuHsil3w,Nevada House of Hose,1015 Sharp Cir,North Las Vegas,NV,89030,36.219728,-115.127725,2.5,3,0,"{'BusinessAcceptsCreditCards': 'True', 'ByAppo...","Hardware Stores, Home Services, Building Suppl...","{'Monday': '7:0-16:0', 'Tuesday': '7:0-16:0', ..."
4,51M2Kk903DFYI6gnB5I6SQ,USE MY GUY SERVICES LLC,4827 E Downing Cir,Mesa,AZ,85205,33.428065,-111.726648,4.5,26,1,"{'BusinessAcceptsCreditCards': 'True', 'ByAppo...","Home Services, Plumbing, Electricians, Handyma...","{'Monday': '0:0-0:0', 'Tuesday': '9:0-16:0', '..."


In [4]:
yelp_original_df.groupby('city')['city'].count().sort_values(ascending=False)

city
Las Vegas           31631
Toronto             20366
Phoenix             20171
Charlotte           10422
Scottsdale           9342
                    ...  
Huntington Beach        1
Huntingdon              1
Queensville             1
Houston                 1
​Gilbert                1
Name: city, Length: 1251, dtype: int64

In [41]:
executable_path = {"executable_path": "/usr/local/bin/chromedriver"}
browser =  Browser("chrome", **executable_path, headless=False)

# iterate through all the rows in the yelp dataframe
    
# visit each restaurant's url
browser.visit("https://www.yelp.com/biz/aljons-pizza-plainsboro")

html = browser.html
soup = BeautifulSoup(html, 'html.parser')

# find the address tag
address = soup.find('address').find_all('span', class_='raw__373c0__3rcx7')

# find address1 and add to list
street_address = address[0].text
for i in range(1, len(address)-1):
    street_address = street_address + address[i].text + " "
print(street_address)

# find the rest of the address
address2 = address[len(address)-1].text

url_link = soup.find('div', class_='stickySidebar__373c0__3PY1o').find('a', class_='lemon--a__373c0__IEZFH').text

if url_link == "Get Directions":
    url_link = ""
print(url_link)

    
browser.quit()


660 Plainsboro Rd
aljonsplainsboro.com
